# Exploring the Security of One-Time and Many-Time Pads

## Overview
This team project investigates the cryptographic security of the one-time pad, known for its perfect security under certain conditions, and explores the implications of reusing a key, referred to as a many-time pad. We will examine the practical aspects of implementing these encryption methods, analyze their vulnerabilities, and develop strategies to exploit these vulnerabilities in a controlled environment. Here is some background story in signal intelligence. https://en.wikipedia.org/wiki/Venona_project
## Objectives
1. Understand One-Time Pad
Examine the concept of the one-time pad, its implementation, and why it is considered perfectly secure.
2. Implement One-Time Pad Encryption and Decryption
Develop a Python program that simulates the encryption and decryption process between two parties.
3. Explore Many-Time Pad
Extend the one-time pad implementation to simulate a many-time pad scenario, where a single key is used to encrypt multiple messages.
4. Cryptanalysis of Many-Time Pad
Design and execute an attack strategy to decrypt messages encrypted with a many-time pad, focusing on exploiting the vulnerabilities introduced by key reuse.

## Problem 1: Understanding One-Time Pad
Tasks:
1. Research the theoretical basis of the one-time pad, including its requirements and operational principles.

## Problem 2: One-Time Pad Implementation
Tasks: the encryption and decryption process between two parties, Alice and Bob.
1. Alice's Program
Should prompt for a message input (plaintext), then display the ciphertext, and save both the ciphertext (in hex) and the key (in hex) in separate files.
2. Bob's Program:
Should read the key and ciphertext from their respective files and display the decrypted plaintext.

In [62]:
from pathlib import Path
import random
import string

# Print formatted output for better readability
def print_formatted(description:str, data:str = None):
    print(f'{description:<50}: {data}')

# Create a mailbox directory to store messages
def create_data_directory(directory_name:str) -> Path:
    file_directory = Path(f'./{directory_name}/')
    file_directory.mkdir(exist_ok=True)
    print_formatted('Using data directory', file_directory)
    return file_directory

# Encode a character
def encode_char(char:str) -> int:
    return ord(char)

def decode_char(num:int) -> str:
    return chr(num)

# Generate a random key of a given length
def generate_key(length:int) -> str:
    key = ''.join(random.choices(string.ascii_lowercase, k=length))
    print_formatted(f'Generate Key of length {length}', key)
    return key

# Vigenere cipher is a method of encrypting alphabetic text by using a simple form of polyalphabetic substitution.
def vigenere_encrypt(plaintext:str, key:str) -> str:
    print_formatted(f'Encrypting plaintext', plaintext)
    encrypted_text = []
    n = len(plaintext)
    for i in range(n):
        x = encode_char(plaintext[i]) + ((encode_char(key[i]) % n) % 26)
        encrypted_text.append(decode_char(x))
    return "".join(encrypted_text)

def vigenere_decrypt(ciphertext:str, key:str) -> str:
    print_formatted(f'Decrypting ciphertext', ciphertext)
    decrypted_text = []
    n = len(ciphertext)
    for i in range(n):
        x = encode_char(ciphertext[i]) - ((encode_char(key[i]) % n) % 26)
        decrypted_text.append(decode_char(x))
    return "".join(decrypted_text)

# For the one-time pad, the key must be as long as the plaintext and is different for each message.
def one_time_pad_encrypt(plaintext:str) -> tuple:
    key = generate_key(len(plaintext))
    return vigenere_encrypt(plaintext, key), key

def one_time_pad_decrypt(ciphertext:str, key:str) -> str:
    return vigenere_decrypt(ciphertext, key)

# Convert string to hex and vice versa
def string_to_hex(text_string:str) -> str:
    return text_string.encode('utf-8').hex()

def hex_to_string(hex_string:str) -> str:
    return bytes.fromhex(hex_string).decode('utf-8')

# Write and read data to/from files
def write_data(directory_path:Path, file_name:str, message:str) -> None:
    data_file = directory_path / f'{file_name}.txt'
    with data_file.open('w') as f:
        f.write(string_to_hex(message))

def read_data(directory_path:Path, file_name:str) -> str:
    data_file = directory_path / f'{file_name}.txt'
    with data_file.open() as f:
        return hex_to_string(f.read())

# Alice and Bob programs
def alice_program(message_name:str, directory_path:Path) -> None:
    # get user input
    plaintext = input('Enter a message: ')
    print_formatted('Plaintext', plaintext)
    # encrypt the message using one-time pad
    ciphertext, key = one_time_pad_encrypt(plaintext)
    print_formatted('Alice sending message (Ciphertext)', ciphertext)
    write_data(directory_path, message_name, ciphertext)
    write_data(directory_path, message_name + '_key', key)

def bob_program(message_name:str, directory_path:Path) -> None:
    ciphertext = read_data(directory_path, message_name)
    key = read_data(directory_path, message_name + '_key')
    # decrypt the message using one-time pad
    message = one_time_pad_decrypt(ciphertext, key)
    print_formatted('Bob received message (Plaintext)', message)

# Test the one-time pad implementation for alice and bob
otp_data_directory = create_data_directory('otp_data')
message_file_name = 'message1'
alice_program(message_file_name, otp_data_directory)
bob_program(message_file_name, otp_data_directory)

Using data directory                              : otp_data
Plaintext                                         : Hello world
Generate Key of length 11                         : vxeelkylcby
Encrypting plaintext                              : Hello world
Alice sending message (Ciphertext)                : Ponnx(wxrvd
Decrypting ciphertext                             : Ponnx(wxrvd
Bob received message (Plaintext)                  : Hello world


## Problem 3: Exploring Many-Time Pad
Tasks: Modify the one-time pad implementation to encrypt multiple messages with the same key, simulating a many-time pad scenario. The purpose of this problem is to see if there are any recognizable patterns by observing the outputs. You can gain insights by changing the plaintexts or the key to verify your findings. These findings would be useful in the next problem.
1. The program should encrypt a list of 10 predefined plaintext messages with a single key, saving the plaintexts, key, and ciphertexts (all in hex) into a file. You can select 10 of your favorite messages. Assume the key is long enough to do encryption to all the 10 messages.

In [66]:
# Many-time pad encryption for multiple plaintexts using the same key
def many_time_pad_encrypt(plaintexts: list, key: str) -> list:
    ciphertexts = []
    for plaintext in plaintexts:
        ciphertexts.append(vigenere_encrypt(plaintext, key))
    return ciphertexts

# Many-time pad decryption for multiple ciphertexts using the same key
def many_time_pad_decrypt(ciphertexts: list, keys: list) -> list:
    plaintexts = []
    for key, ciphertext in zip(keys, ciphertexts):
        plaintexts.append(vigenere_decrypt(ciphertext, key))
    return plaintexts

# Write and read many-time pad data to/from files
# The format of the files is key,plaintext,ciphertext
# The key, plaintext, and ciphertext are in hex format
def write_many_time_pad_data(directory_path: Path, file_name:str, key: str, plaintexts:list, ciphertexts: list) -> None:
    data_file = directory_path / f'{file_name}.txt'
    with data_file.open('w') as f:
        for plaintext, ciphertext in zip(plaintexts, ciphertexts):
            f.write(f'{string_to_hex(key)},{string_to_hex(plaintext)},{string_to_hex(ciphertext)}\n')

def read_many_time_pad_data(directory_path: Path, file_name:str) -> tuple:
    keys = []
    plaintexts = []
    ciphertexts = []
    data_file = directory_path / f'{file_name}.txt'
    with data_file.open() as f:
        for line in f:
            k, p, c = line.split(',')
            keys.append(hex_to_string(k.strip()))
            plaintexts.append(hex_to_string(p.strip()))
            ciphertexts.append(hex_to_string(c.strip()))
    return keys, plaintexts, ciphertexts

# Many-time pad encryption and decryption for multiple messages
# 10 even longer messages to test
mtp_messages = [
    'hello world, how are you doing today?',
    'python programming is fun and challenging',
    'data science is the future of technology',
    'machine learning is a subset of artificial intelligence',
    'deep neural network is a type of machine learning',
    'cryptography is the practice and study of secure communication',
    'information security is the protection of information',
    'computer science is the study of computation',
    'software engineering is the application of engineering to software',
    'data analytics is the science of analyzing raw data'
    ]

mtp_key = generate_key(100)
mtp_ciphertexts = many_time_pad_encrypt(mtp_messages, mtp_key)
many_time_pad_data_directory = create_data_directory('mtp_data')
many_time_pad_file_name = 'many_time_pad_data'
write_many_time_pad_data(many_time_pad_data_directory, many_time_pad_file_name, mtp_key, mtp_messages, mtp_ciphertexts)
mtp_keys, mpt_plaintexts, mtp_ciphertexts = read_many_time_pad_data(many_time_pad_data_directory, many_time_pad_file_name)
for i in range(len(mtp_messages)):
    print_formatted('Message', i+1)
    print_formatted('Many-Time Pad Key', mtp_keys[i])
    print_formatted('Many-Time Pad Plaintext', mpt_plaintexts[i])
    print_formatted('Many-Time Pad Ciphertext', mtp_ciphertexts[i])

mtp_plaintexts = many_time_pad_decrypt(mtp_ciphertexts, mtp_keys)
for plaintext in mtp_plaintexts:
    print_formatted('Many-Time Pad Decrypted Plaintext', plaintext)

Generate Key of length 100                        : pbmdhksdovziflcrgxfgftbtvthuaffdxsxsawhgmmmwjjjnjdjmbodmutnmpukoeofgyqfafcijmavvtqvglbfgoqthiiqugjka
Encrypting plaintext                              : hello world, how are you doing today?
Encrypting plaintext                              : python programming is fun and challenging
Encrypting plaintext                              : data science is the future of technology
Encrypting plaintext                              : machine learning is a subset of artificial intelligence
Encrypting plaintext                              : deep neural network is a type of machine learning
Encrypting plaintext                              : cryptography is the practice and study of secure communication
Encrypting plaintext                              : information security is the protection of information
Encrypting plaintext                              : computer science is the study of computation
Encrypting plaintext                      